In [ ]:
import psycopg
from pgvector.psycopg import register_vector

In [ ]:
# connect to database d with user postgres and password postgres

conn = psycopg.connect(
    dbname='oscar',
    user='oscar',
    password='oscar',
    host='localhost',
    autocommit=True
)
# register_vector(conn)

In [ ]:
cur = conn.cursor()
cur.execute("DROP EXTENSION IF EXISTS vector CASCADE")
cur.execute("CREATE EXTENSION vector")
register_vector(conn)

In [ ]:
import numpy as np
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS test")
cur.execute("CREATE TABLE test (id serial PRIMARY KEY, vec vector(3))")
cur.execute("INSERT INTO test (vec) VALUES (%s)", (np.array([1, 2, 3]),))
cur.execute("SELECT * FROM test")
print(cur.fetchall())

In [ ]:
# set the pinecone api key
cur = conn.cursor()
cur.execute("ALTER SYSTEM SET pinecone.api_key TO 'your-api-key-here'")
cur.execute("SHOW pinecone.api_key")
print(cur.fetchall())

In [ ]:
# create a pinecone index
cur = conn.cursor()
import json
basic_spec = {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}}
cur.execute("CREATE INDEX test_index ON test USING pinecone (vec) with (spec = '%s')" % json.dumps(basic_spec))


In [ ]:
# set debugging level to debug1
import pprint
def notice_handler(notice):
                print("Received notice:", notice.message_primary)
                # user pprint to print the notice as a dictionary
                # pprint(notice.__reduce__())
# if not any notice handlers
conn.notice_handlers = []
conn.add_notice_handler(notice_handler)
for handler in conn.notice_handlers:
    print(handler)

cur = conn.cursor()
debug_level = "notice"
cur.execute("ALTER SYSTEM SET client_min_messages TO '%s'" % debug_level)
cur.execute("SELECT pg_reload_conf()")
cur.execute("SHOW client_min_messages")
print(cur.fetchall())

In [ ]:
# query the index
cur = conn.cursor()
cur.execute("SET enable_seqscan = off")
cur.execute("SELECT * FROM test ORDER BY vec <-> %s LIMIT 1", (np.array([1, 2, 3]),))
print(cur.fetchall())

In [ ]:
# batch query
for _ in range(10):
    cur.execute("SELECT * FROM test ORDER BY vec <-> %s LIMIT 1", (np.random.rand(3),))
    print(cur.fetchall())

In [ ]:
import asyncpg
async def async_query(pool):
    async with pool.acquire() as conn:
        print("connection acquired from pool")
        # conn.execute("SELECT * FROM test ORDER BY vec <-> '[1,2,3]' LIMIT 1")
        print(await conn.fetch("SELECT * FROM test ORDER BY vec <-> '[1,2,3]' LIMIT 1"))

# Run the async_query function
async def run_async_query():
    pool = await asyncpg.create_pool(user='oscar', password='oscar', database='oscar', host='localhost', min_size=2, max_size=20)
    await async_query(pool)
    await pool.close()

async def run_async_queries():
    pool = await asyncpg.create_pool(user='oscar', password='oscar', database='oscar', host='localhost', min_size=2, max_size=20)
    await asyncio.gather(*[async_query(pool) for _ in range(10)])
    await pool.close()

await run_async_queries()